

Copyright (c) University of Strasbourg. All Rights Reserved.

<a href="https://colab.research.google.com/github/CAMMA-public/ai4surgery/blob/master/ai4surgery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<a href="http://camma.u-strasbg.fr/">
<img src="https://raw.githubusercontent.com/CAMMA-public/ai4surgery/master/figs/data_camma_logo_tr.png" width="100" align="left"/>
</a>
</div>

## <h1><center>AI for surgery: Hands-on material</center></h1>


# **Artificial Intelligence in Surgery: An AI Primer for Surgical Practice**

**Neural Networks and Deep Learning**

_Deepak Alapatt and Pietro Mascagni, Vinkle Srivastav, Nicolas Padoy_


# Introduction

This hands-on lab demonstrates how designing and training a neural network works in practice.  It is designed as a supplement to the **Neural Networks and Deep learning** chapter of the book **Artificial Intelligence in Surgery: An AI Primer for Surgical Practice**.

Specifically, in this notebook we will develop and train a simple neural network for surgical tool detection in laparoscopic images. With the advent of advanced deep learning software libraries like tensorflow, many of the steps involved in working with neural networks have been abstracted out allowing for a much simpler process.

To use the notebook, you will need to first log in to your google account.

# Basics

Below is a code cell that assigns the value 3 and 4 to variables *a* and *b*, respectively. Then the function *max* assigns the maximum value of *a* and *b* to a variable *c*. The values stored in all three variables are then printed to the screen.

In [ ]:
# This is a cell to code
# Comments following a # can be added and won't affect the code flow

# Assign the value 3 to the variable a and the value 4 to the variable 4
a = 3     
b = 4

# Assign the maximum value of a and b to c                                                
c = max(a, b)                     

# Return the values of a, b and c                        
print(a, b, c)                                            

To run the code in the above cell, select it with a click and then either press the *play* button on the left of the cell, or use the shortcut "Command/Ctrl+Enter". To edit the code, just type directly into the cell and re-run the cell to see the result.


# Imports

We import some libraries which contain functions that are useful for building and visualizing neural networks. For those unfamiliar with programming, just execute the cell below and move on to the next section.

In [ ]:
# Install needed libraries

!pip install -q tensorflow==2.2.0
!pip install -q matplotlib==3.2.2
!pip install -q numpy==1.18.5

# Import the required libraries

# Tensorflow contains functions needed to build and train neural networks
import tensorflow as tf
# Matlpotlib is a plotting library to help visualize our inputs and outputs data
import matplotlib.pyplot as plt
# Numpy is a library to simplify operations on matrices and other 
# mathematical objects
import numpy as np
# Random is a library to help generate random numbers, make random choices, etc
import random

print("Libraries successfully imported!")

# Dataset

The neural network will take an input image containg a surgical tool tip and classify the image as showing either a **grasper**, a **hook**, a **clipper** or a **scissor**. For this purpose, we will use [Cholec-tinytools](https://s3.unistra.fr/camma_public/github/ai4surgery/cholec-tinytools.zip), a dataset of images of surgical tool tips generated from **Cholec80**. **Cholec80** is a widely used dataset containing 80 laparoscopic cholecystectomy videos annotated with surgical phases and tool presence released by the [CAMMA research group](http://camma.u-strasbg.fr/) (University of Strasbourg, France).


![](https://raw.githubusercontent.com/CAMMA-public/ai4surgery/master/figs/dataset.png)

In [ ]:
# Download and extract the tool tip dataset from an online repository 

DATA_URL = (
    'https://s3.unistra.fr/camma_public/github/ai4surgery/cholec-tinytools.zip'
)
path = tf.keras.utils.get_file('cholec-tinytools.zip', DATA_URL, extract=True)
  
#Stores the dataset in the variable "path"
path = path.strip('.zip')                                     

print("Dataset successfully extracted!")

Note: The code above may take a few minutes to be executed.

In [ ]:
# Define the dataset characteristics

# ~60% of the dataset for training
TRAINING_SET_SIZE = 1200      

# ~10% of the dataset for validation
VALIDATION_SET_SIZE = 200  

# ~30% of the dataset for testing                              
TEST_SET_SIZE = 599              

# The names of surgical tools we want to classify
CLASS_NAMES = ['grasper', 'hook', 'scissor', 'clipper']

# The numbers of surgical tools (i.e. classes) we want to classify     
NUMBER_CLASSES = 4                   

# Each input image will have a resolution of 86x128 pixels                        
IMAGE_SIZE = (86, 128)                        

# RGB IMAGES are represented with 3 channels (red, blue, green)                
CHANNELS = 3
   
# Number of images per batch                                               
BATCH_SIZE = 16                                              

print("Dataset characteristics defined!")

Each pixel of an RGB image can be represented as a combination of red, green and blue values each between 0-255.
![](https://raw.githubusercontent.com/CAMMA-public/ai4surgery/master/figs/rgb_decomp.png)


In [ ]:
# Preprocess the images in the dataset

# The training images are scaled down (i.e. normalized) 
# to the range 0-1 by dividing by 255
# To artificially augment the training set, we additionally use variants of  
# each image generated by randomnly rotating it by 0-20 degrees
train_preprocessing = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, rotation_range=20
)
train_set = tf.keras.preprocessing.image.DirectoryIterator(
    path +'/train',
    image_data_generator=train_preprocessing,
    batch_size=BATCH_SIZE,
    target_size=IMAGE_SIZE,
    classes=CLASS_NAMES,
    shuffle=True,
)

# The validation and test images are similarly scaled down (i.e. normalized) 
# to the range 0-1 by dividing by 255
validation_test_preprocessing = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
validation_set = tf.keras.preprocessing.image.DirectoryIterator(
    path +'/validation',
    image_data_generator=validation_test_preprocessing,
    batch_size=BATCH_SIZE,
    classes=CLASS_NAMES,
    target_size=IMAGE_SIZE
)

test_set = tf.keras.preprocessing.image.DirectoryIterator(
    path +'/test',
    image_data_generator=validation_test_preprocessing,
    batch_size=BATCH_SIZE,
    classes=CLASS_NAMES,
    target_size=IMAGE_SIZE
)

print("\n Dataset successfully preprocessed!")

Preview 4 random images from the training set

In [ ]:
# Define the figure using a matplotlib.pyplot function
fig, axes = plt.subplots(1,4, figsize=(15, 15))               
random_batch =  random.randint(0, len(train_set)-1)
random_images, random_labels = train_set[random_batch]

# For loops are used to iterate over a sequence, in this case each 
# image being plotted
for image_number, axs in enumerate(axes):                     
    img = random_images[image_number]
    axs.imshow(img)
    axs.axis("off")
    axs.set_title(CLASS_NAMES[np.argmax(random_labels[image_number])])

Rerun the cell above to visualize the different appearances of the 4 types of tool tips in the dataset.

# Convolutions



<div>
<center>
<img src="https://raw.githubusercontent.com/CAMMA-public/ai4surgery/master/figs/convolution.gif" width="350" /> </center>
</div>

Here we will apply filters for edge detection to demonstrate the kind of information a single convolutional filter can extract from an image.

In [ ]:
# Read a sample image
img = plt.imread(path + "/train/hook/7013_16226.png")

# Convert the image to grayscale
img_gray = tf.image.rgb_to_grayscale(img)

# Visualize result 
fig = plt.figure(figsize=(15,5))

# Display the original input and it's grayscale version 
fig.add_subplot(1, 2, 1); plt.imshow(img);
plt.title("Original Input"); plt.axis("off")

fig.add_subplot(1, 2, 2); plt.imshow(img_gray.numpy()[..., 0], 'gray');
plt.title("Grayscale Input"); plt.axis("off")
plt.show()

Kernels for edge detection

<div>
<center>
<img src="https://raw.githubusercontent.com/CAMMA-public/ai4surgery/master/figs/sobel_filter.png" width="600" /> </center>
</div>


The above kernels or filters contain [known weight values](https://en.wikipedia.org/wiki/Sobel_operator) for horizontal and vertical edges detection.


In [ ]:
# Define filters for edge detection

# Horizontal filter
horizontal_edge_detector = tf.constant(
    [[1,2,1],[0,0,0],[-1,-2,-1]], dtype=tf.float32, shape=(3, 3, 1, 1)
)

# Vertical filter
vertical_edge_detector = tf.constant(
    [[1,0,-1],[2,0,-2],[1,0,-1]], dtype=tf.float32, shape=(3, 3, 1, 1)
)

# Display filters
# We use ".numpy()" to display the result as a matrix
print('Horizontal filter')
print(horizontal_edge_detector[:,:,0,0].numpy())

print('Vertical filter')
print(vertical_edge_detector[:,:,0,0].numpy())

print("Edge detectors tensors successfully defined")

Since most tensorflow functions are designed to take multiple batches of input images at a time, we first convert the input image to a 4 dimensional matrix as shown below.

In [ ]:
# Expand to have 4 dimensional (4D) matrix
# This is done because the tensorflow library expects the input to the 
# convolution to be in the shape Batch x Height x Width x Channels
# In this case Batch = 1

input_4d = tf.expand_dims(img_gray, 0)

Apply a convolution for horizontal edge detection on the grayscale image

In [ ]:
# Convolution on the image with horizontal filter
# The filter is shifted of 1 pixel (strides = 1) at every step, 
# zero padding is applied such the output will have the "SAME" spatial 
# dimensions as the input
conv_h = tf.nn.conv2d(input=input_4d, filters=horizontal_edge_detector, strides=1, padding="SAME") 

# The filter will output high absolute values (maybe positive or negative) 
# when it detects horizontal edges
horizontal_edge_img = tf.abs(conv_h)

# Visualize result 
fig = plt.figure(figsize=(15,5))

# Display input
fig.add_subplot(1, 2, 1); plt.imshow(input_4d.numpy()[0,:,:,0], 'gray');
plt.title("Input Gray"); plt.axis("off")

# Display detected horizontal edges
fig.add_subplot(1, 2, 2); plt.imshow(horizontal_edge_img.numpy()[0,:,:,0], 'gray');
plt.title("Horizontal edges"); plt.axis("off")

plt.show()

Similarly for vertical edge detection

In [ ]:
conv_v = tf.nn.conv2d(input=input_4d, filters=vertical_edge_detector, strides=1, padding="SAME") 
vertical_edge_img = tf.abs(conv_v)

# Visualize result 
fig = plt.figure(figsize=(15,5))

# Display input
fig.add_subplot(1, 2, 1); plt.imshow(input_4d.numpy()[0,:,:,0], 'gray');
plt.title("Input Gray"); plt.axis("off")

# Display detected vertical edges
fig.add_subplot(1, 2, 2); plt.imshow(vertical_edge_img.numpy()[0,:,:,0], 'gray');
plt.title("Vertical edges"); plt.axis("off")

plt.show()

In [ ]:
# The detected edges in the horizontal and vertical direction can then be 
# combined to identify all the edges in the image 

# Create a single output by adding the detected vertical and horizontal edges
edge_detections = horizontal_edge_img + vertical_edge_img

In [ ]:
# Visualize result 
fig = plt.figure(figsize=(30,5))

# Display input
fig.add_subplot(1, 5, 1); plt.imshow(input_4d.numpy()[0,:,:,0], 'gray');
plt.title("Input Gray"); plt.axis("off")

# Display detected horizontal edges
fig.add_subplot(1, 5, 2); plt.imshow(horizontal_edge_img.numpy()[0,:,:,0], 'gray');
plt.title("Horizontal edges"); plt.axis("off")

# Display detected horizontal edges
fig.add_subplot(1, 5, 3); plt.imshow(vertical_edge_img.numpy()[0,:,:,0], 'gray'); 
plt.title("Vertical edges"); plt.axis("off")

# Display combined horizontal and vertical edges
fig.add_subplot(1, 5, 4); plt.imshow(edge_detections.numpy()[0,:,:,0], 'gray'); 
plt.title("Combined"); plt.axis("off")

plt.show()

When building neural networks, filter weights will be learned during training to extract and aggregate information relevant to the target task.

## Convolution using `tf.keras`



Keras is a high-level interface library running on top of TensorFlow and other machine learning frameworks. It was designed to be more intuitive and user-friendly to enable fast experimentation with deep neural networks.

In [ ]:
# Again, we expand the dimensions so that our input is in the shape
# Batch x Height x Width x Channels

input_4d = tf.expand_dims(img, 0)

In [ ]:
# Use keras for convolution

# Random convolution operation (i.e. untrained kernel weights)
conv_img = tf.keras.layers.Conv2D(
    filters=3, kernel_size=(3, 3),
    strides=1, padding="SAME",
    input_shape=(1, 86, 128, 3))(input_4d)
conv_img = conv_img.numpy()[0]

# Normalize so that our output values lie within a valid range for display
# i.e. 0-1
min, max = conv_img.min(), conv_img.max()
conv_img = (conv_img - min)/(max-min)

# Visualize result
fig = plt.figure(figsize=(15,5))

# Display input
fig.add_subplot(1, 2, 1); plt.imshow(img);plt.title("Input"); 
plt.axis("off")

# Display convolution output
fig.add_subplot(1, 2, 2); plt.imshow(conv_img);plt.title("Output");
plt.axis("off")

plt.show()

Here the kernel weights are randomly initiated and not trained to extract any particular feature, hence the output changes at every execution. This demonstrates the kind of information that can get filtered out or focused on by a single convolution.


## Pooling


<div>
<center>
<img src="https://raw.githubusercontent.com/CAMMA-public/ai4surgery/master/figs/pooling.png" width="600" /> </center>
</div>


Pooling layers apply mathematical operations such as averaging and maximum to reduce the size of the inputs. Such operations are applied using sliding filters, similar to the convolutional filters described above. 

We will now apply max pooling and average pooling to visualize how they differently affect an input. Feel free to explore other filter sizes.

In [ ]:
# Experimenting with max pooling filter sizes

# Change the value below to see how filter size affects the output
filter_size = 5

# Max Pooling using filter_size x filter_size (default: 5x5)
# The filter is shifted with a stride of 2 in both the horizontal
# and vertical directions.
maxpooled_img =  tf.keras.layers.MaxPool2D(
    pool_size=(filter_size, filter_size), strides=2)(input_4d)

# Visualize the max pooling output
fig = plt.figure()
plt.imshow(maxpooled_img.numpy()[0]);
plt.title("max pool {}".format(filter_size)); plt.axis("off")

plt.show()

In [ ]:
# Experimenting with max pooling filter sizes

# Max Pooling using 5x5 kernel, strides of 5
maxpooled_img5x5 =  tf.keras.layers.MaxPool2D(
    pool_size=(5, 5), strides=5)(input_4d)

# Max Pooling using 10x10 kernel, strides of 10
maxpooled_img10x10 = tf.keras.layers.MaxPool2D(
    pool_size=(10, 10), strides=10)(input_4d)

# Max Pooling using 20x20 kernel, strides of 20
maxpooled_img20x20 =  tf.keras.layers.MaxPool2D(
    pool_size=(20, 20), strides=20)(input_4d)

# Visualize max pooling output
fig = plt.figure(figsize=(10,5))

fig.add_subplot(1, 3, 1); plt.imshow(maxpooled_img5x5.numpy()[0]);
plt.title("max pool 5x5")

fig.add_subplot(1, 3, 2); plt.imshow(maxpooled_img10x10.numpy()[0])
plt.title("max pool 10x10")

fig.add_subplot(1, 3, 3); plt.imshow(maxpooled_img20x20.numpy()[0]);
plt.title("max pool 20x20")

plt.show()

In [ ]:
# Experimenting with average pooling filter sizes

# Average Pooling using 5x5 kernel, stride of 5
avgpooled_img5x5 = tf.keras.layers.AveragePooling2D(
    pool_size=(5, 5), strides=5)(input_4d)

# Average Pooling using 10x10 kernel, stride of 10
avgpooled_img10x10 = tf.keras.layers.AveragePooling2D(
    pool_size=(10, 10), strides=10)(input_4d)

# Average Pooling using 20x20 kernel, strides of 20
avgpooled_img20x20 = tf.keras.layers.AveragePooling2D(
    pool_size=(20, 20), strides=20)(input_4d)

# Visualize average pooling output
fig = plt.figure(figsize=(10,5))

fig.add_subplot(1, 3, 1); plt.imshow(avgpooled_img5x5.numpy()[0]);
plt.title("avg pool 5x5")

fig.add_subplot(1, 3, 2); plt.imshow(avgpooled_img10x10.numpy()[0]);
plt.title("avg pool 10x10")

fig.add_subplot(1, 3, 3); plt.imshow(avgpooled_img20x20.numpy()[0]);
plt.title("avg pool 20x20")

plt.show()



In both average and max pooling, there is a trade-off between image(or feature) size and spatial detail.

# Tool Classification

It is now time to build a neural network for surgical tool classification. The classification model is defined below as a sequential stack of layers (picturized below).


<div>
<center>
<img src="https://raw.githubusercontent.com/CAMMA-public/ai4surgery/master/figs/cnn.png" width="750" /> </center>
</div>

In [ ]:
# Defining the neural network architecture
model = tf.keras.Sequential()

# Adding a convolution with 16 5x5 filters followed by a ReLU activation
model.add(tf.keras.layers.Conv2D(
    filters=16, kernel_size=5, activation="relu", input_shape=(86, 128, 3))
)
# Adding max pooling over 5x5 patches of the previous layers output
model.add(tf.keras.layers.MaxPooling2D(pool_size=(5,5)))     

# Adding a second convolution with 32 3x3 filters followed by a ReLU activation                   
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")) 

# Adding a layer to flatten the multidimensional (HxWxC) input
model.add(tf.keras.layers.Flatten())                           

# Adding a fully connected layer with 4096 outputs followed by a ReLU activation                 
model.add(tf.keras.layers.Dense(units=4096, activation="relu"))                 

# Adding a fully connected layer with 2048 outputs followed by a ReLU activation
model.add(tf.keras.layers.Dense(units=2048, activation="relu"))       

# Adding a fully connected layer with 4 outputs followed by a softmax activation          
model.add(tf.keras.layers.Dense(units=NUMBER_CLASSES, activation="softmax"))    

print("Neural network architecture successfully defined!")

Notes:
*   The output of the convolutional layer is a 3 dimensional HxWxC matrix for each element of the batch. This should be flattened to a single row of H*W*C elements before applying a fully connected layers.
*   Fully connected layers are sometimes called dense layers because each output (here referred to as units) is densely connected to the previous layer.
*   The model ends with a fully connected layer with 4 outputs and a softmax activation. This activation function is used to convert the output of the previous fully connected layer into a vector of 4 probabilities that sum to one, i.e. the probability of each surgical instrument to be in the image.


In [ ]:
# Defining the optimization method, a loss function and a metric

# The optimization method is stochasitc gradient descent(SGD)
opt = tf.keras.optimizers.SGD(learning_rate=0.01) 

# We specify the expected input shape to build our model 
# batch size x height x width x channels                   
model.build([1, 86, 128, 3])

# Categorical cross entropy is commonly used loss for classification problems
model.compile(
    optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']
)

# model.summary generates a neat summary listing the number of parameters
# in the entire network and per layer
model.summary()                                                                     

Note: **None** indicates that any batch size can be passed through the same network using the same architecture.

Let's run the untrained network on a single image to see what the network input and output look like.

In [ ]:
# Read image
img = plt.imread(path + "/train/hook/7013_16226.png")
print('Input:')
plt.imshow(img)
plt.show()

# Expand to have 4 dimensional (4D) image tensor
input_4d = tf.expand_dims(img, 0)

# Uses the untrained model defined above to predict  
prediction = model.predict(input_4d)                                   
print('Output:')
print(prediction[0])
print(CLASS_NAMES)

The network predicts 4 probability values corresponding to every considered class. Note that since we used a softmax activation in our last layer, the 4 probabilities sum to 1. Right now, the predictions are random; we will train the network to learn parameters to make better predictions.

In [ ]:
# Training our neural network
# The model will iterate over the training data 15 times (i.e. epochs)
history = model.fit(train_set, validation_data=validation_set, epochs=15)       

# Plot the validation and test results for each training epoch
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

The above graphic plots the neural network accuracy on the training set and on the validation set over training epochs. When the two lines converge training is effective, when those diverge (i.e. higher accuracy on the training set then on the validation set) the model is overfitting to the training data. An overfitted model will fail to generalize to unseen data (i.e. will perform poorly on test data). 

In [ ]:
 model.evaluate(test_set)

The **evaluate** function returns the model loss and accuracy (first and second number in squared brackets, respectively) on the test set. The goal is to get the loss close to 0 and accuracy close to 1 (100%).

Finally, we visualize the results on a few random images from the test set.

In [ ]:
# Define the figure using a matplotlib.pyplot function
fig, axes = plt.subplots(1,4, figsize=(15, 15))               
random_batch =  random.randint(0, len(test_set)-1)
random_images, random_labels = test_set[random_batch]

predictions = model.predict(random_images)

# For loops are used to iterate over a sequence, in this case each 
# image being plotted
for image_number, axs in enumerate(axes):                     
    img = random_images[image_number]
    axs.imshow(img)
    axs.axis("off")
    prediction = CLASS_NAMES[np.argmax(predictions[image_number])]
    label = CLASS_NAMES[np.argmax(random_labels[image_number])]
    axs.set_title("Predicted Class {} \n True Class {}".format(prediction, label))

# Design your surgical tool classifier

Here's a copy of the same model to play around with. Your aim shuld be to design a network architecture and pick its corresponding hyperparameters to maximize the validation accuracy.

*   See what effect increasing and decreasing the number of epochs has on training
*   Play around with the learning rate to see what's the optimal value
*   Try changing the parameters (number of filters, kernel size, etc...)
*   Try out different activation functions ("tanh", "relu", etc...)
*   Try adding or removing layers


In [ ]:
my_model = tf.keras.Sequential()
my_model.add(
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation="relu")
)
my_model.add(tf.keras.layers.MaxPooling2D(pool_size=(5,5)))
my_model.add(
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")
)
my_model.add(tf.keras.layers.Flatten())
my_model.add(tf.keras.layers.Dense(units=4096, activation="relu"))
my_model.add(tf.keras.layers.Dense(units=2048, activation="relu"))
my_model.add(tf.keras.layers.Dense(units=NUMBER_CLASSES, activation="softmax"))

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
my_model.compile(
    optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']
)

history = my_model.fit(train_set, validation_data=validation_set, epochs=15)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
my_model.evaluate(test_set)